## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку. 

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Общая информация показала, что в колонках days_employed и total_income есть пропущенные строки. Их количество равно, соответственно, эти люди никогда не работали официально. 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
df['days_employed'] = df['days_employed'].fillna(0)
df['total_income'] = df['total_income'].fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Так как был сделан вывод, что люди не работали, заполняем пропуски нулями.

### Замена типа данных

In [3]:
df['days_employed'] = df['days_employed'].astype('int').abs()
df['total_income'] = df['total_income'].astype('int')
df['children'] = df['children'].abs()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Произвели замену отрицательных чисел на положительные целые. Использованы методы astype и abs. 

### Обработка дубликатов и удаление не отражающих действительность значений 

In [4]:
df = df.drop(df[df['days_employed'] > 36500].index) #убрали инплейс и предопределили переменную 
mean_dob = df['dob_years'].mean()
df['dob_years'].replace('0', mean_dob)
df.drop_duplicates(inplace=True)
df.drop(df[df['children'] > 5].index, inplace = True)


df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17957 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          17957 non-null  int64 
 1   days_employed     17957 non-null  int64 
 2   dob_years         17957 non-null  int64 
 3   education         17957 non-null  object
 4   education_id      17957 non-null  int64 
 5   family_status     17957 non-null  object
 6   family_status_id  17957 non-null  int64 
 7   gender            17957 non-null  object
 8   income_type       17957 non-null  object
 9   debt              17957 non-null  int64 
 10  total_income      17957 non-null  int64 
 11  purpose           17957 non-null  object
dtypes: int64(7), object(5)
memory usage: 1.8+ MB


### Вывод

Были удалены полные дубликаты данных, которые, вероятно, по-ошибке были повторно внесены. В строки, где возраст равнялся нулю, поставили средний возраст по таблице. Удалены строки, где количество дней стажа составляло больше условно 36500 дней, тогда как данные не отражают действительность и не могут быть использованы для дальнейшего анализа. 

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()

df['purpose_lemmas'] = list(map(m.lemmatize, df['purpose']))
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'операции с жильем', 'образование',
       'на проведение свадьбы', 'покупка жилья для семьи',
       'покупка недвижимости', 'покупка коммерческой недвижимости',
       'сыграть свадьбу', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку своего автомобиля',
       'на покупку подержанного автомобиля',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'операции с коммерческой недвижимостью', 'свой автомобиль',
       'сделка с автомобилем', 'профильное образование',
       'высшее об

### Вывод

Провели лемматизацию столбца целей выдачи кредита. Добавили полученный результат в качестве дополнительного столбца. Выявили с помощью unique() ключевые слова, по которым будем категоризировать.

### Категоризация данных

In [6]:
#будем категоризировать по уровню дохода и присвоим каждому уровню свой индекс.


def total_income_index(income_sum):
    if income_sum == 0:
        return '0' #доход отсутствует
    if income_sum > 0 and income_sum < 15000:
        return '1' #очень низкий доход
    if income_sum >= 15000 and income_sum < 30000:
        return '2' #низкий доход
    if income_sum >= 30000 and income_sum < 60000:
        return '3' #средний доход
    if income_sum >= 60000 and income_sum < 100000:
        return '4' #доход выше среднего
    if income_sum >= 100000 and income_sum < 150000:
        return '5' #высокий доход
    if income_sum >= 150000 and income_sum < 250000:
        return '6' #очень высокий доход
    if income_sum >= 250000 and income_sum < 500000:
        return '7' #сверхвысокий доход 1
    if income_sum >= 500000 and income_sum < 1000000:
        return '8' #сверхвысокий доход 2
    if income_sum >= 1000000:
        return '9' #запредельный доход
     
df['total_income_index'] = df['total_income'].apply(total_income_index)


#категоризируем по основным ключевым словам из столбца цели выдачи кредита. Присваем каждой категории индекс

def purpose_id(words):
    if 'свадьба' in words:
        return '1'
    if 'недвижимость' in words:
        return '2'
    if 'автомобиль' in words:
        return '3'
    if 'жилье' in words:
        return '4'
    if 'образование' in words:
        return '5'
    
df['purpose_lemmas_id'] = df['purpose_lemmas'].apply(purpose_id)

### Вывод

Провели категоризацию по уровню дохода и различным целям кредита. Присвоили индексы, по которым будем группировать.

Уровень дохода определен исходя из допущения, что у нас местная валюта (рубли). Соответственно, категоризация условно определена по покупательной способности денег.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:

df.groupby('children')['debt'].sum() / df.groupby('children')['debt'].count()

children
0    0.081804
1    0.094212
2    0.094349
3    0.080247
4    0.100000
5    0.000000
Name: debt, dtype: float64

### Вывод

Сгруппировали людей, просрочивших кредит, по количеству детей в семье. Затем поделили на общее количество получивших кредит людей в определенной группе, чтобы выявить процентное соотношение. 

Таким образом, было выявлено, что количество просрочивших кредит варьируется от <b>8% до 10%</b>, независимо от количества детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
df.groupby('family_status_id')['debt'].sum() / df.groupby('family_status_id')['debt'].count()


# 0 - женат / замужем
# 1 - гражданский брак
# 2 - вдовец / вдова
# 3 - в разводе
# 4 - не женат / не замужем

family_status_id
0    0.079609
1    0.099580
2    0.065539
3    0.073367
4    0.104904
Name: debt, dtype: float64

### Вывод

Сгруппировали людей, просрочивших кредит, по семейному положению. Затем поделили на общее количество получивших кредит людей, чтобы выявить процентное соотношение. 

Было выявлено, что из всех групп наиболее часто не возвращают кредит вовремя лица, находящиеся в <b>гражданском браке</b> или <b>неженатые / незамужние</b>: около <b>10%</b> случаев.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
df.groupby('total_income_index')['debt'].sum() / df.groupby('total_income_index')['debt'].count()

# тыс. 
#0 - доход отсутствует - 0 
#1 - очень низкий доход - до 15
#2 - низкий доход - 15-30
#3 - средний доход - 30-60
#4 - доход выше среднего - 60-100
#5 - высокий доход - 100-150
#6 - очень высокий доход - 150-250
#7 - сверхвысокий доход 1 - 250-500
#8 - сверхвысокий доход 2 - 500-1000
#9 - запределельный доход - >1000

total_income_index
0    0.080531
2    0.000000
3    0.069977
4    0.096062
5    0.092859
6    0.086790
7    0.071213
8    0.067416
9    0.080000
Name: debt, dtype: float64

### Вывод

В ходе исследования было выявлено, что категория людей из группы, которая наиболее часто не отдает кредит в срок, имеет доход от <b>60 тыс. до 150 тыс.</b> (от <b>9,2 до 9,6%</b> случаев). В среднюю категорию риска входят лица <b>не имеющие дохода</b>, лица с доходом от <b>150</b> до <b>250</b> тыс. или с доходом <b>свыше 1 млн</b>. В наименьшую категорию риска из группы входят люди с доходом от <b>30 тыс. до 60 тыс.</b> и от <b>250 тыс. до 1 млн.</b>. Отдельно стоит отметить категорию лиц с доходом до <b>15 тыс.</b>, которые вообще не взяли кредит и от <b>15 до 30 тыс.</b>, которые не имели просрочек по платежам. 

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
df.groupby('purpose_lemmas_id')['debt'].sum() / df.groupby('purpose_lemmas_id')['debt'].count()

#1 - свадьба
#2 - недвижимость
#3 - автомобиль
#4 - жилье
#5 - образование


purpose_lemmas_id
1    0.084016
2    0.080188
3    0.100028
4    0.073216
5    0.097758
Name: debt, dtype: float64

### Вывод

В ходе исследования выявлено, что люди, берущие кредит на <b>автомобили и образование</b> наиболее часто просрочивают кредит (<b>10%</b> всех случаев). Кредиты на <b>жилье</b> просрочиваются <b>реже всех</b> (около <b>7,3%</b>). Остальные категории кредита (<b>свадьба, недвижимость</b>) имеют риск быть просроченными с вероятностью от <b>8%</b> до <b>8,4%</b>. 

### Шаг 4. Общий вывод

<i>В ходе исследования было установлено, что наиболее благоприятный заемщик:</i><br/> 
1) находится в <b>разводе</b><br/>
2) имеет доход от <b>15 тыс.</b> до <b>60 тыс.</b> или от <b>250 тыс. включительно до миллиона.</b><br/>
3) берет кредит на <b>жилье.</b><br/>

<i>Заемщик представляющий наибольший определенный риск:</i><br/>
1) находится в <b>гражданском браке</b> или <b>не женат</b> / <b>не замужем</b>.<br/>
2) имеет доход от <b>60 тыс. включительно до 150 тыс.</b><br/>
3) берет кредит на <b>автомобиль или образование</b>.<br/>

<i>прим.</i> было также выявлено, что количество детей <b>не имеет определяющего значения</b>